<a href="https://colab.research.google.com/github/nabilneal/DataScience/blob/main/BERTFINETUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
# nRowsRead = None # specify 'None' if want to read whole file
# # labeled_data.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
# df0 = pd.read_csv(r"train.csv", delimiter=',', nrows = nRowsRead)
# df01 = pd.read_csv(r"test_with_solutions.csv", delimiter=',', nrows = nRowsRead)
# df0.dataframeName = 'train.csv'
# df01.dataframeName = 'test_with_solutions.csv'
# nRow, nCol = df0.shape
# nRow1, nCol1 = df01.shape
# print('There are {} rows and {} columns'.format(nRow, nCol))
# print('There are {} rows and {} columns'.format(nRow1, nCol1))

In [ ]:
# c=df0['Insult']
# df0.rename(columns={'Comment' : 'text',
#                    'Insult' : 'category'},
#                     inplace=True)

# a=df0['text']

# b=df0['category'].map({1: 'offensive_language',0 : 'non_offensive_language'})

# df= pd.concat([a,c], axis=1)
# df.rename(columns={'Insult' : 'label'},
#                     inplace=True)
# df

In [ ]:
# def clean_data(dataframe):
# #replace URL of a text
#     dataframe['text'] = dataframe['text'].str.replace('RT', ' ')
#     dataframe['text'] = dataframe['text'].str.replace(r'\\n', '')
#     dataframe['text'] = dataframe['text'].str.replace(r'\\x', '')
#     dataframe['text'] = dataframe['text'].str.replace(r'\\u', '')
#     dataframe['text'] = dataframe['text'].str.replace('a0', '')
#     dataframe['text'] = dataframe['text'].str.replace('c2', '')
#     dataframe['text'] = dataframe['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
#     dataframe['text'] = dataframe['text'].str.replace('@(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
#     dataframe['text'] = dataframe['text'].str.replace('[^\w\s#@/:%.,_-]', ' ')
#     dataframe['text'] = dataframe['text'].str.replace(' s ', 's ')
#     dataframe['text'] = dataframe['text'].str.replace(' t ', 't ')
#     dataframe['text'] = dataframe['text'].str.replace(' T ', 'T ')
#     dataframe['text'] = dataframe['text'].str.replace(' re ', ' are ')


# clean_data(df)

In [ ]:
nRowsRead = None # specify 'None' if want to read whole file
# df02 = pd.read_csv(r"tweet2.csv", delimiter=',', nrows = nRowsRead)
# df02.dataframeName = 'tweet2.csv'
df01 = pd.read_csv(r"tweet2.csv", delimiter=',', nrows = nRowsRead)
df01.dataframeName = 'tweet2.csv'
# df00 = pd.read_csv(r"tweet.csv", delimiter=',', nrows = nRowsRead)
# df00.dataframeName = 'tweet.csv'
nRow1, nCol1 = df01.shape
# nRow2, nCol2 = df02.shape
print('There are {} rows and {} columns'.format(nRow1, nCol1))
# print('There are {} rows and {} columns'.format(nRow2, nCol2))

There are 24782 rows and 4 columns


In [ ]:
df = pd.concat([df01['text'],df01['label']],axis = 1)
# df2 = pd.concat([df02['text'],df02['label']],axis = 1)
# df0 = pd.concat([df00['text'],df00['label']],axis = 1)
# d_f = pd.concat([df1, df2], ignore_index=True)
# df = pd.concat([d_f, df0], ignore_index=True)

In [ ]:
 # Grouping data by label
df.groupby('label').count()
text = df.text.values
labels = df.label.values
df.groupby('label').count()

,text
label,
0,20619
1,4163


In [ ]:

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [ ]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒═══════════╤═════════════╕
│ Tokens    │   Token IDs │
╞═══════════╪═════════════╡
│ 10        │        2184 │
├───────────┼─────────────┤
│ sex       │        3348 │
├───────────┼─────────────┤
│ positions │        4460 │
├───────────┼─────────────┤
│ ,         │        1010 │
├───────────┼─────────────┤
│ ,         │        1010 │
├───────────┼─────────────┤
│ na        │        6583 │
├───────────┼─────────────┤
│ ##w       │        2860 │
├───────────┼─────────────┤
│ this      │        2023 │
├───────────┼─────────────┤
│ what      │        2054 │
├───────────┼─────────────┤
│ u         │        1057 │
├───────────┼─────────────┤
│ call      │        2655 │
├───────────┼─────────────┤
│ fucking   │        8239 │
├───────────┼─────────────┤
│ the       │        1996 │
├───────────┼─────────────┤
│ soul      │        3969 │
├───────────┼─────────────┤
│ outta     │       24955 │
├───────────┼─────────────┤
│ bitch     │        7743 │
╘═══════════╧═════════════╛


In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids'])
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
token_id[9]

tensor([  101,  1024,  7570, 27982,  2421,  1024,  3554,  3814,  2213,  7743,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])

In [ ]:

def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │         101 │                1 │
├──────────┼─────────────┼──────────────────┤
│ had      │        2018 │                1 │
├──────────┼─────────────┼──────────────────┤
│ to       │        2000 │                1 │
├──────────┼─────────────┼──────────────────┤
│ get      │        2131 │                1 │
├──────────┼─────────────┼──────────────────┤
│ that     │        2008 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ho       │        7570 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##e      │        2063 │                1 │
├──────────┼─────────────┼──────────────────┤
│ shit     │        4485 │                1 │
├──────────┼─────────────┼──────────────────┤
│ off      │        2125 │                1 │
├──────────┼─────────────┼──────────────────┤
│ my       │        2026 │        

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx],
                          attention_masks[train_idx],
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx],
                        attention_masks[val_idx],
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(),
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()
# model.cpu()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 3

for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids,
                             token_type_ids = None,
                             attention_mask = b_input_mask,
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []
    val_f1 = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids,
                              token_type_ids = None,
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)
        # Update f1-score only when f1 !=0; ignore nan
        if b_precision != 'nan' and b_recall != 'nan': b_f1 = 2* (b_precision * b_recall)/ (b_precision + b_recall)
        if b_f1 > 0: val_f1.append(b_f1)


    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')
    print('\t - Validation F1-score: {:.4f}\n'.format(sum(val_f1)/len(val_f1)) if len(val_f1)>0 else '\t - Validation F1-score: NaN')



Epoch:   0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-43-1c17c84938ce>:66: RuntimeWarning: invalid value encountered in double_scalars
  if b_precision != 'nan' and b_recall != 'nan': b_f1 = 2* (b_precision * b_recall)/ (b_precision + b_recall)
Epoch:  33%|███▎      | 1/3 [01:07<02:15, 67.74s/it]


	 - Train loss: 0.1358
	 - Validation Accuracy: 0.9514
	 - Validation Precision: 0.7844
	 - Validation Recall: 0.9662
	 - Validation Specificity: 0.9484
	 - Validation F1-score: 0.8722



Epoch:  67%|██████▋   | 2/3 [02:14<01:07, 67.20s/it]


	 - Train loss: 0.1005
	 - Validation Accuracy: 0.9585
	 - Validation Precision: 0.8550
	 - Validation Recall: 0.9104
	 - Validation Specificity: 0.9686
	 - Validation F1-score: 0.8852



Epoch: 100%|██████████| 3/3 [03:21<00:00, 67.25s/it]


	 - Train loss: 0.0653
	 - Validation Accuracy: 0.9567
	 - Validation Precision: 0.8230
	 - Validation Recall: 0.9371
	 - Validation Specificity: 0.9609
	 - Validation F1-score: 0.8821



In [ ]:
# new_sentence = 'You deserve to die'
# new_sentence = 'He is such a dumbfuck.'
# new_sentence = 'It was a fun day'
new_sentence = 'Don\'t worry your pretty little head.'
# new_sentence = 'You’re a girl, you should know better.'


# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Hateful' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 0 else 'Non-Hate'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  Don't worry your pretty little head.
Predicted Class:  Non-Hate


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
import re
sent = []
# new_sentence = 'You deserve to die'
# new_sentence = 'He is such a dumbfuck.'
# new_sentence = 'It was a fun day'
new_sentence = 'Don\'t worry your pretty little head.'
# new_sentence = 'You’re a girl, you should know better.'

df_svm = pd.read_csv(r"wrongpredic.csv")

df_svm['Text'] = df_svm['Text'].apply(lambda x: str(x) if str(x).endswith(('.', '!', '?')) else str(x) + '.')
df_svm['sentence_count'] = df_svm['Text'].apply(lambda x: len(re.findall('[.!?]', str(x))))
df_svm['word_count'] = df_svm['Text'].apply(lambda x: len(str(x).split(" ")))
ts_df = df_svm[df_svm['sentence_count'] == 2]

# print(df_svm)
# sent = [ 'You deserve to die.', 'You’re a girl, you should know better.','Let’s Kill Jews, and let’s kill them for fun. #killjews',
#         'It was a fun day.', 'I think you have a lot of potential, and if you work on your public speaking skills, you could really excel in your career.',
#         "How was your weekend? Did you do anything fun?"]
sent =ts_df['Text'].tolist()


for i in sent:
    # We need Token IDs and Attention Mask for inference on the new sentence
    test_ids = []
    test_attention_mask = []

    # Apply the tokenizer
    encoding = preprocessing(i, tokenizer)

    # Extract IDs and Attention Mask
    test_ids.append(encoding['input_ids'])
    test_attention_mask.append(encoding['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)

    # Forward pass, calculate logit predictions
    with torch.no_grad():
      output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

    prediction = 'Hateful' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 0 else 'Non-Hate'
    label = ts_df[ts_df['Text']==i]['Label']
    # print(label)
    # print("size: ", len(label))
    # print(label.values[0].item())
    if(len(label)==1):
        actual_l = 'Hateful' if int(label.item()) == 0 else 'Non-Hate'

    elif(len(label)>1):
        actual_l = 'Hateful' if int(label.values[:1].item()) == 0 else 'Non-Hate'


    print(actual_l)
    print('Input Sentence: ', i)
    print('Predicted Class: ', prediction)
    print("\n")

Hateful
Input Sentence:  LOVE Debra and Eva, and Sheri and Emilie, Such beautiful souls. #mkr Will Sheri and Eva ever kiss.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  Did Jezebel or Marcotte or Valenti write about Carnell, I forgot. They don't care.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  That face Kat just pulled was hideous. #mkr.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  Now who has had the smile wiped off their face. Kat you are a biatch, #mkr.
Predicted Class:  Hateful


Hateful
Input Sentence:  The Ezidis were out in the open with no shelter for 6 month because of Daeshbags. So who cares.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  Earthly tyrants want to be feared because for them fear is control and obedience., The writer of the Quran was unsophisticated.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  is awesome, if you don't know her., Write the truth about feminism.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  I w

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Hateful
Input Sentence:  If they think that dessert is refined and beautiful, now wonder they think they are pretty. #MKR.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  More men actually die from DV once suicide as a result is factored in. People tend to miss.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  What black and white, terrorist scum, There is zero evidence presented. Are you a madrassa graduate.
Predicted Class:  Hateful


Non-Hate
Input Sentence:  And no one said only women should stick together. Non-rapist men can also stick together with their, women, friends.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  That face Kat just pulled was hideous. #mkr.
Predicted Class:  Non-Hate


Non-Hate
Input Sentence:  Note,, is obvs not sexist. Retweet needs a, sincere or sarcastic,  option.
Predicted Class:  Non-Hate


Hateful
Input Sentence:  This is an idiotic study made with an objective in mind., Terrorism is not included in the murder rate.
Predicted Class:  

In [ ]:
import re
sent = []
# new_sentence = 'You deserve to die'
# new_sentence = 'He is such a dumbfuck.'
# new_sentence = 'It was a fun day'
new_sentence = 'Don\'t worry your pretty little head.'
# new_sentence = 'You’re a girl, you should know better.'

df_svm = pd.read_csv(r"wrongpredic.csv")

df_svm['Text'] = df_svm['Text'].apply(lambda x: str(x) if str(x).endswith(('.', '!', '?')) else str(x) + '.')
df_svm['sentence_count'] = df_svm['Text'].apply(lambda x: len(re.findall('[.!?]', str(x))))
df_svm['word_count'] = df_svm['Text'].apply(lambda x: len(str(x).split(" ")))
ts_df = df_svm[df_svm['sentence_count'] == 2]

# print(df_svm)
# sent = [ 'You deserve to die.', 'You’re a girl, you should know better.','Let’s Kill Jews, and let’s kill them for fun. #killjews',
#         'It was a fun day.', 'I think you have a lot of potential, and if you work on your public speaking skills, you could really excel in your career.',
#         "How was your weekend? Did you do anything fun?"]
sent =ts_df['Text'].tolist()


for i in sent:
    # We need Token IDs and Attention Mask for inference on the new sentence
    test_ids = []
    test_attention_mask = []

    # Apply the tokenizer
    encoding = preprocessing(i, tokenizer)

    # Extract IDs and Attention Mask
    test_ids.append(encoding['input_ids'])
    test_attention_mask.append(encoding['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)

    # Forward pass, calculate logit predictions
    with torch.no_grad():
      output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

    prediction = 'Hateful' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 0 else 'Non-Hate'

    print('Input Sentence: ', i)
    print('Predicted Class: ', prediction)

Input Sentence:  LOVE Debra and Eva, and Sheri and Emilie, Such beautiful souls. #mkr Will Sheri and Eva ever kiss.
Predicted Class:  Non-Hate
Input Sentence:  Did Jezebel or Marcotte or Valenti write about Carnell, I forgot. They don't care.
Predicted Class:  Non-Hate
Input Sentence:  That face Kat just pulled was hideous. #mkr.
Predicted Class:  Non-Hate
Input Sentence:  Now who has had the smile wiped off their face. Kat you are a biatch, #mkr.
Predicted Class:  Hateful
Input Sentence:  The Ezidis were out in the open with no shelter for 6 month because of Daeshbags. So who cares.
Predicted Class:  Non-Hate
Input Sentence:  Earthly tyrants want to be feared because for them fear is control and obedience., The writer of the Quran was unsophisticated.
Predicted Class:  Non-Hate
Input Sentence:  is awesome, if you don't know her., Write the truth about feminism.
Predicted Class:  Non-Hate
Input Sentence:  I wonder who slept on this in tent, , Celine - I m talking to you. #Burp, #mkr.
P

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(



Input Sentence:  What black and white, terrorist scum, There is zero evidence presented. Are you a madrassa graduate.
Predicted Class:  Hateful
Input Sentence:  And no one said only women should stick together. Non-rapist men can also stick together with their, women, friends.
Predicted Class:  Non-Hate
Input Sentence:  That face Kat just pulled was hideous. #mkr.
Predicted Class:  Non-Hate
Input Sentence:  Note,, is obvs not sexist. Retweet needs a, sincere or sarcastic,  option.
Predicted Class:  Non-Hate
Input Sentence:  This is an idiotic study made with an objective in mind., Terrorism is not included in the murder rate.
Predicted Class:  Non-Hate
Input Sentence:  Kybar was attacked by Mohammed., The whole excuse for the attack is a fabrication for which no evidence exists.
Predicted Class:  Non-Hate
Input Sentence:  have you even seen a women kick a football. I thinks it's probably genetic.
Predicted Class:  Non-Hate
Input Sentence:  Exactly., #Isam is a cult of death centered a